In [1]:
%%time
import boto3
# SUBO MI CODIGO PYTHON EN UNA RUTA ESPECIFICA

BUCKET_NAME = 'attemp-glue'
AWS_DEFAULT_ACL = 'public-read'
REGION = 'us-west-1'
path_in_pc = '/home/rmg/STORAGE/tests_courses_research/practice/GLUE/campaing/code_up_campaing.py'
path_s3 = 'code_up_campaing.py'

client = boto3.client('s3')
s3 = boto3.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

if not bucket.creation_date:
    s3.create_bucket(
        ACL=AWS_DEFAULT_ACL,
        Bucket=BUCKET_NAME,
        CreateBucketConfiguration={'LocationConstraint': REGION}
    )
    
client.upload_file(
    path_in_pc,
    BUCKET_NAME,
    path_s3,
    ExtraArgs={
        'ContentType': 'text/x-python',
        'ACL': AWS_DEFAULT_ACL
    }
)

CPU times: user 269 ms, sys: 16 ms, total: 285 ms
Wall time: 1.44 s


In [2]:
%%time
# PARA EJECUTAR UNA SOLA VEZ
# Creando el job GLUE
job_name = 'copy-json-glue'

client_glue = boto3.client('glue')
response_glue = client_glue.create_job(
    Name=job_name,
    Description='Prueba de copias csv en flujo GLUE',
    Role='AdminGlue',
    ExecutionProperty={
        'MaxConcurrentRuns': 3
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': 's3://{}/{}'.format(BUCKET_NAME, path_s3)
    }
)
response_glue

CPU times: user 77.9 ms, sys: 8.15 ms, total: 86 ms
Wall time: 994 ms


In [3]:
# RUN JOB GLUE
response_run = client_glue.start_job_run(
    JobName=job_name
)
response_run

{'JobRunId': 'jr_b34b2f6f0a3db01005a8fe727566521cd104f68ab3615763472a6847741c7ed5',
 'ResponseMetadata': {'RequestId': '3abb955b-57e5-11e9-b6c6-37da6c5ab0ad',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Apr 2019 20:56:05 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3abb955b-57e5-11e9-b6c6-37da6c5ab0ad'},
  'RetryAttempts': 0}}

In [4]:
job_run_id = response_run.get('JobRunId', '')
job_run_id

'jr_b34b2f6f0a3db01005a8fe727566521cd104f68ab3615763472a6847741c7ed5'

In [5]:
import time
status = 'RUNNING'

while status == 'RUNNING':
    response_status = client_glue.get_job_run(
        JobName=job_name,
        RunId = job_run_id
    )
    status = response_status.get('JobRun', {}).get('JobRunState', {})
    print(status, end=' ')
    if status != 'RUNNING':
        print("\n ", response_status)
        break
        
    time.sleep(10)

RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING RUNNING SUCCEEDED 
  {'JobRun': {'Id': 'jr_b34b2f6f0a3db01005a8fe727566521cd104f68ab3615763472a6847741c7ed5', 'Attempt': 0, 'JobName': 'copy-json-glue', 'StartedOn': datetime.datetime(2019, 4, 5, 15, 56, 5, 804000, tzinfo=tzlocal()), 'LastModifiedOn': datetime.datetime(2019, 4, 5, 15, 58, 23, 632000, tzinfo=tzlocal()), 'CompletedOn': datetime.datetime(2019, 4, 5, 15, 58, 23, 632000, tzinfo=tzlocal()), 'JobRunState': 'SUCCEEDED', 'PredecessorRuns': [], 'AllocatedCapacity': 10, 'ExecutionTime': 115, 'Timeout': 2880, 'LogGroupName': '/aws-glue/jobs'}, 'ResponseMetadata': {'RequestId': '9035e90b-57e5-11e9-a8a0-f711dd56b6ea', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 05 Apr 2019 20:58:29 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '376', 'connection': 'keep-alive', 'x-amzn-requestid': '9035e90b-57e5-11e9-a8a0-f711dd56b6ea'}, 'RetryAttempts': 0}}
